In [19]:
%%capture
%pip install azure-core
%pip install azure-ai-formrecognizer
%pip install tabulate
%pip install fuzzywuzzy

In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import pandas as pd

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
# endpoint = os.getenv("FORM_RECOGNIZER_ENDPOINT")
# key = os.getenv("FORM_RECOGNIZER_KEY")
endpoint ="https://scraper.cognitiveservices.azure.com/"
key ="8b2877ef2b52444886bb09e0c5be84e5"

print(key)
print(endpoint)
# iniitalize the client
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)



8b2877ef2b52444886bb09e0c5be84e5
https://scraper.cognitiveservices.azure.com/


In [5]:
base_path = "/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/"

companies = [
    "Tanso Technologies GmbH_München.pdf",
    "Tacto Technology GmbH_München.pdf",
    "simpleclub GmbH_München.pdf",
    "sento GmbH_München.pdf",
    "Zavvy GmbH_München.pdf"


]

json_array = []

for company in companies:
    startup_name, _ = os.path.splitext(company)  # Remove the file extension
    file_path = os.path.join(base_path, company)  # Create the full file path
    json_object = {
        "startup_name": startup_name,
        "file_path": file_path
    }
    json_array.append(json_object)

# Now `json_array` is an array of JSON objects
print(json_array)

[{'startup_name': 'Tanso Technologies GmbH_München', 'file_path': '/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/Tanso Technologies GmbH_München.pdf'}, {'startup_name': 'Tacto Technology GmbH_München', 'file_path': '/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/Tacto Technology GmbH_München.pdf'}, {'startup_name': 'simpleclub GmbH_München', 'file_path': '/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/simpleclub GmbH_München.pdf'}, {'startup_name': 'sento GmbH_München', 'file_path': '/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/sento GmbH_München.pdf'}, {'startup_name': 'Zavvy GmbH_München', 'file_path': '/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/Zavvy GmbH_München.pdf'}]


In [6]:
def get_table_data(result):
    rows = []

    # get header row
    header_row = {}
    try:
        for cell in result.tables[0].cells:
            if(cell.kind != "columnHeader"):
                continue

            # if row index is 0, add cell content to header row
            if cell.row_index == 0:
                for i in range(cell.column_index, cell.column_index + cell.column_span):
                    header_row[i] = cell.content
                
            # if row index is larger than 0, append cell content to existing header row for the correct column
            elif cell.row_index > 0:
                for i in range(cell.column_index, cell.column_index + cell.column_span):
                    header_row[i] += "\n" + cell.content
    except:
        pass

    # append header row to rows list
    rows.append(header_row)

    # get table content
    for table_idx, table in enumerate(result.tables):
        
        row = {}
        row_index = 0
     
        try:
            for cell in table.cells:
                # skip the first header row of the first table
                if table_idx == 0 and cell.kind == "columnHeader":
                    continue

                # append constructed row if previous row is complete
                if cell.row_index > row_index:
                    if row != {}:
                        rows.append(row)
                    row = {}
                    row_index = cell.row_index
            

                # add cell to row - if cell spans multiple columns, add the cell content to every column
                for i in range(cell.column_index, cell.column_index + cell.column_span):
                    # row[i] = { header_row[i]: cell.content }
                    row[i] = cell.content
        except:
            pass
        rows.append(row)

    # convert list of dicts to dataframe
    df = pd.DataFrame(rows[1:])
    df.columns = rows[0].values()
    return df


In [7]:

# def open_file(file_path):
#     #open pdf from file path
#     with open(file_path, "rb") as f:
#         buffer = f.read()
#     return buffer
import PyPDF2
import io
chunk_size = 2
   
def open_file(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        df_list = pd.DataFrame() 
        for i in range(0, len(reader.pages), chunk_size):
            writer = PyPDF2.PdfWriter()

            for j in range(i, min(i + chunk_size, len(reader.pages))):
                writer.add_page(reader.pages[j])

            pdf_bytes = io.BytesIO()
            writer.write(pdf_bytes)
            pdf_bytes.seek(0)

            # Now `pdf_bytes` is a file-like object containing the PDF data.
            # This can be sent to an API as follows:
            response = analyze_PDF(pdf_bytes)
            table = get_table_data(response)
            # Check the response
            df_list=df_list.append(table)
            print(df_list)
            # Clear the writer for the next chunk of pages
            writer = PyPDF2.PdfWriter()

    return df_list

def analyze_PDF(buffer):
    poller = document_analysis_client.begin_analyze_document("prebuilt-layout",buffer)
    result = poller.result()
    return result

In [18]:
buffer = open_file("/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/The Exploration Company GmbH_München.pdf")



/var/folders/nw/rzxfctl55nj95y0twwbqxrm40000gn/T/ipykernel_27689/1367750351.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list=df_list.append(table)


                                      Gesellschafter  \
0  Artur Koop, geboren am 16. Juni 1991, wohnhaft...   
1  Sebastien Reichstadt, geboren am 6. Oktober 19...   
2                                     Gesellschafter   
3  Johannes Reijneveld, geboren am 4. Juni 1986, ...   
4    The Exploration Company GmbH - eigene Anteile -   
5  Priyanka Das, geboren am 3. Oktober 1991, wohn...   
6  Hélène Huby, geboren am 20. Februar 1978, wohn...   
7  Pierre Vinet, geboren am 20. April 1975, wohnh...   
8  André Lanata, geboren am 10. Oktober 1961, woh...   
9    Christian Dargnat, geboren am 13. Oktober 1965,   

            Lfd. Nr(n). der Geschäfts- anteile  \
0  14.551-17.850 35.354-35.398 96.433 - 96.460   
1                  17.851-20.350 35.399-35.420   
2           Lfd. Nr(n). der Geschäfts- anteile   
3  20.351-22.850 35.421-35.465 96.461 - 96.474   
4                                22.851-24.450   
5                                35.466-35.478   
6       251-13.050 14.351-14.550 

/var/folders/nw/rzxfctl55nj95y0twwbqxrm40000gn/T/ipykernel_27689/1367750351.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list=df_list.append(table)


                                       Gesellschafter  \
0   Artur Koop, geboren am 16. Juni 1991, wohnhaft...   
1   Sebastien Reichstadt, geboren am 6. Oktober 19...   
2                                      Gesellschafter   
3   Johannes Reijneveld, geboren am 4. Juni 1986, ...   
4     The Exploration Company GmbH - eigene Anteile -   
5   Priyanka Das, geboren am 3. Oktober 1991, wohn...   
6   Hélène Huby, geboren am 20. Februar 1978, wohn...   
7   Pierre Vinet, geboren am 20. April 1975, wohnh...   
8   André Lanata, geboren am 10. Oktober 1961, woh...   
9     Christian Dargnat, geboren am 13. Oktober 1965,   
0                       wohnhaft in Paris, Frankreich   
1   AURA SAS mit dem Sitz in Paris, Frankreich (Re...   
2   Deep Impact Investment Holding GmbH mit dem Si...   
3   Temaris et associés SAS mit dem Sitz in Paris,...   
4   Aperitus Limited mit dem Sitz in Santa Venera,...   
5       Nils Bernhardt, geboren am 26. Dezember 1994,   
6                              

/var/folders/nw/rzxfctl55nj95y0twwbqxrm40000gn/T/ipykernel_27689/1367750351.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list=df_list.append(table)


...

/var/folders/nw/rzxfctl55nj95y0twwbqxrm40000gn/T/ipykernel_27689/1367750351.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list=df_list.append(table)


                                       Gesellschafter  \
0   Artur Koop, geboren am 16. Juni 1991, wohnhaft...   
1   Sebastien Reichstadt, geboren am 6. Oktober 19...   
2                                      Gesellschafter   
3   Johannes Reijneveld, geboren am 4. Juni 1986, ...   
4     The Exploration Company GmbH - eigene Anteile -   
..                                                ...   
5                                        10437 Berlin   
6   Sebastien Ayton, geboren am 06.06.1986, wohnha...   
7   Lear Javiv, geboren am 25.10.1985, wohnhaft in...   
8   Interface Capital I GmbH & Co. KG mit dem Sitz...   
9   Izzat Holdings Ltd. mit dem Sitz in Tortola, B...   

             Lfd. Nr(n). der Geschäfts- anteile  \
0   14.551-17.850 35.354-35.398 96.433 - 96.460   
1                   17.851-20.350 35.399-35.420   
2            Lfd. Nr(n). der Geschäfts- anteile   
3   20.351-22.850 35.421-35.465 96.461 - 96.474   
4                                 22.851-24.450   
..       

/var/folders/nw/rzxfctl55nj95y0twwbqxrm40000gn/T/ipykernel_27689/1367750351.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list=df_list.append(table)


                                       Gesellschafter  \
0   Artur Koop, geboren am 16. Juni 1991, wohnhaft...   
1   Sebastien Reichstadt, geboren am 6. Oktober 19...   
2                                      Gesellschafter   
3   Johannes Reijneveld, geboren am 4. Juni 1986, ...   
4     The Exploration Company GmbH - eigene Anteile -   
..                                                ...   
1   Marc Postel-Vinay, geboren am 17.01.1986, wohn...   
2   Habert Dassault Finance SAS mit dem Sitz in Pa...   
3   Schlumberger Investment Services B.V. mit dem ...   
4                                      Gesellschafter   
5          Anschrift: Parkstraat 83, 2514 JG Den Haag   

             Lfd. Nr(n). der Geschäfts- anteile  \
0   14.551-17.850 35.354-35.398 96.433 - 96.460   
1                   17.851-20.350 35.399-35.420   
2            Lfd. Nr(n). der Geschäfts- anteile   
3   20.351-22.850 35.421-35.465 96.461 - 96.474   
4                                 22.851-24.450   
..       

/var/folders/nw/rzxfctl55nj95y0twwbqxrm40000gn/T/ipykernel_27689/1367750351.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list=df_list.append(table)


In [19]:
buffer

,Gesellschafter,Lfd. Nr(n). der Geschäfts- anteile,Nominalwert des einzelnen Geschäftsan- teils,Prozen- tualer Anteil des einzelnen Ge- schäftsan- teils am Stammka- pital,Zwischensumme (mit prozentualer Angabe),Summe des Nominal- werts aller Geschäftsan- teile,Gesamt- umfang der Betei- ligung des Gesell- schafters am Stammka- pital,Veränderungen
0,"Artur Koop, geboren am 16. Juni 1991, wohnhaft...",14.551-17.850 35.354-35.398 96.433 - 96.460,"1,00 €","0,001 %","3.300 € (3,271%) 45 € (0,045%) 28 € (0,028%)","3.373,00 €","3,34%",Ifd. Nr. 96.433 - 96.460 erhalten durch Kapita...
1,"Sebastien Reichstadt, geboren am 6. Oktober 19...",17.851-20.350 35.399-35.420,"1,00 €","0,001 %","22 € (0,022%) 2.500 € (2,478%)","2.522,00 €","2,50%",Änderung Wohnort
2,Gesellschafter,Lfd. Nr(n). der Geschäfts- anteile,Nominalwert des einzelnen Geschäftsan- teils,Prozen- tualer Anteil des einzelnen Ge- schäft...,Zwischensumme (mit prozentualer Angabe),Summe des Nominal- werts aller Geschäftsan- teile,Gesamt- umfang der Betei- ligung des Gesell- s...,Veränderungen
3,"Johannes Reijneveld, geboren am 4. Juni 1986, ...",20.351-22.850 35.421-35.465 96.461 - 96.474,"1,00 €","0,001 %","2.500 € (2,478%) 45 €(0,045%) 14 € (0,014%)","2.559,00 €","2,54%",Ifd. Nr. 96.461 - 96.474 erhalten durch Kapita...
4,The Exploration Company GmbH - eigene Anteile -,22.851-24.450,"1,00 €","0,001 %",,1.600 €,"1,59%",
...,...,...,...,...,...,...,...,...
1,"Marc Postel-Vinay, geboren am 17.01.1986, wohn...",99.275 - 99.618,"1,00 €","0,001 %",,"344,00 €","0,34%",erhalten durch Kapitaler- höhungsbeschluss vom...
2,Habert Dassault Finance SAS mit dem Sitz in Pa...,99.619 - 100.598,"1,00 €","0,001 %",,"980,00 €","0,97%",erhalten durch Kapitaler- höhungsbeschluss vom...
3,Schlumberger Investment Services B.V. mit dem ...,100.599 - 101.124,"1,00 €","0,001 %",,"526,00 €","0,52%",erhalten durch Kapitaler- höhungsbeschluss vom...
4,Gesellschafter,Lfd. Nr(n). der Geschäfts- anteile,Nominalwert des einzelnen Geschäftsan- teils,Prozen- tualer Anteil des einzelnen Ge- schäft...,Zwischensumme (mit prozentualer Angabe),Summe des Nominal- werts aller Geschäftsan- teile,Gesamt- umfang der Betei- ligung des Gesell- s...,Veränderungen


In [12]:
str(buffer.to_csv)



'<bound method NDFrame.to_csv of                                        Gesellschafter  \\\n0   Artur Koop, geboren am 16. Juni 1991, wohnhaft...   \n1   Sebastien Reichstadt, geboren am 6. Oktober 19...   \n2                                      Gesellschafter   \n3   Johannes Reijneveld, geboren am 4. Juni 1986, ...   \n4     The Exploration Company GmbH - eigene Anteile -   \n..                                                ...   \n1   Marc Postel-Vinay, geboren am 17.01.1986, wohn...   \n2   Habert Dassault Finance SAS mit dem Sitz in Pa...   \n3   Schlumberger Investment Services B.V. mit dem ...   \n4                                      Gesellschafter   \n5          Anschrift: Parkstraat 83, 2514 JG Den Haag   \n\n             Lfd. Nr(n). der Geschäfts- anteile  \\\n0   14.551-17.850 35.354-35.398 96.433 - 96.460   \n1                   17.851-20.350 35.399-35.420   \n2            Lfd. Nr(n). der Geschäfts- anteile   \n3   20.351-22.850 35.421-35.465 96.461 - 96.474   \n4      

In [240]:
buffer = open_file("/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/eGym GmbH_München.pdf")
print(buffer)

2024-01-17 17:56:35,462:INFO - Request URL: 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout:analyze?stringIndexType=unicodeCodePoint&api-version=2023-07-31'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/octet-stream'
    'Accept': 'application/json'
    'x-ms-client-request-id': '5f88da7a-b559-11ee-92d9-daa58ad9cf8a'
    'User-Agent': 'azsdk-python-ai-formrecognizer/3.3.2 Python/3.11.1 (macOS-12.5-arm64-arm-64bit)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
A body is sent with the request
2024-01-17 17:56:35,886:INFO - Response status: 202
Response headers:
    'Content-Length': '0'
    'Operation-Location': 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/d948120c-9402-4ba4-a454-d84215865bc5?api-version=2023-07-31'
    'x-envoy-upstream-service-time': '67'
    'apim-request-id': 'd948120c-9402-4ba4-a454-d84215865bc5'
    'Strict-

                        Vor- und Nachna- me / Firma  Geburtsdatum / HR-Nummer  \
0      Philipp Roesch- Schlanderer Investments GmbH    AG München, HRB 189933   
1                                               NaN                       NaN   
2                                               NaN                       NaN   
3                                               NaN                       NaN   
4                                               NaN                       NaN   
5                                               NaN                       NaN   
6   Sauter Technolo- gies UG (haftungs- beschränkt)    AG München, HRB 189862   
7                                               NaN                       NaN   
8                                               NaN                       NaN   
9                                     Mirablau GmbH    AG München, HRB 139517   
10                                              NaN                       NaN   
11                          

2024-01-17 17:56:46,339:INFO - Request URL: 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/f35796d5-3b8f-4a01-a032-b18dede0f6d6?api-version=2023-07-31'
Request method: 'GET'
Request headers:
    'x-ms-client-request-id': '62f407c0-b559-11ee-92d9-daa58ad9cf8a'
    'User-Agent': 'azsdk-python-ai-formrecognizer/3.3.2 Python/3.11.1 (macOS-12.5-arm64-arm-64bit)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
No body was attached to the request
2024-01-17 17:56:46,437:INFO - Response status: 200
Response headers:
    'Content-Length': '127349'
    'Content-Type': 'application/json; charset=utf-8'
    'x-envoy-upstream-service-time': '48'
    'apim-request-id': '0985de0c-86fe-4315-98de-0176367de2be'
    'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload'
    'x-content-type-options': 'nosniff'
    'x-ms-region': 'Germany West Central'
    'Date': 'Wed, 17 Jan 2024 16:56:46 GMT'
/var/folders/nw/rzxfc

                     Vor- und Nachna- me / Firma Geburtsdatum / HR-Nummer  \
0   Philipp Roesch- Schlanderer Investments GmbH   AG München, HRB 189933   
1                                            NaN                      NaN   
2                                            NaN                      NaN   
3                                            NaN                      NaN   
4                                            NaN                      NaN   
..                                           ...                      ...   
35                                           NaN                      NaN   
36                                           NaN                      NaN   
37                                           NaN                      NaN   
38                                           NaN                      NaN   
39                                           NaN                      NaN   

                 Wohnort / Sitz         Lfd. Nr.  \
0   Grünwald, Landkreis

2024-01-17 17:56:51,825:INFO - Request URL: 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/5b469884-eec8-48a5-bac2-343235bc44ff?api-version=2023-07-31'
Request method: 'GET'
Request headers:
    'x-ms-client-request-id': '66390390-b559-11ee-92d9-daa58ad9cf8a'
    'User-Agent': 'azsdk-python-ai-formrecognizer/3.3.2 Python/3.11.1 (macOS-12.5-arm64-arm-64bit)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
No body was attached to the request
2024-01-17 17:56:51,900:INFO - Response status: 200
Response headers:
    'Content-Length': '106'
    'Content-Type': 'application/json; charset=utf-8'
    'x-envoy-upstream-service-time': '18'
    'apim-request-id': 'b8765b27-03cf-4d0a-8265-8e731239d31e'
    'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload'
    'x-content-type-options': 'nosniff'
    'x-ms-region': 'Germany West Central'
    'Date': 'Wed, 17 Jan 2024 16:56:51 GMT'
2024-01-17 17:56:56,907:

...

2024-01-17 17:57:19,489:INFO - Request URL: 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/5d90d4fe-2c28-4cf0-91c0-b8d398a64f3c?api-version=2023-07-31'
Request method: 'GET'
Request headers:
    'x-ms-client-request-id': '76b3da92-b559-11ee-92d9-daa58ad9cf8a'
    'User-Agent': 'azsdk-python-ai-formrecognizer/3.3.2 Python/3.11.1 (macOS-12.5-arm64-arm-64bit)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
No body was attached to the request
2024-01-17 17:57:19,631:INFO - Response status: 200
Response headers:
    'Content-Length': '70067'
    'Content-Type': 'application/json; charset=utf-8'
    'x-envoy-upstream-service-time': '40'
    'apim-request-id': '33d8379c-39c5-4876-a18f-16bff8a3ae15'
    'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload'
    'x-content-type-options': 'nosniff'
    'x-ms-region': 'Germany West Central'
    'Date': 'Wed, 17 Jan 2024 16:57:19 GMT'
/var/folders/nw/rzxfct

                     Vor- und Nachna- me / Firma Geburtsdatum / HR-Nummer  \
0   Philipp Roesch- Schlanderer Investments GmbH   AG München, HRB 189933   
1                                            NaN                      NaN   
2                                            NaN                      NaN   
3                                            NaN                      NaN   
4                                            NaN                      NaN   
..                                           ...                      ...   
2                                            NaN                      NaN   
3                                            NaN                      NaN   
4                                            NaN                      NaN   
5                                            NaN                      NaN   
6                                            NaN                      NaN   

                 Wohnort / Sitz         Lfd. Nr.  \
0   Grünwald, Landkreis

2024-01-17 17:57:36,711:INFO - Request URL: 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout:analyze?stringIndexType=unicodeCodePoint&api-version=2023-07-31'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/octet-stream'
    'Accept': 'application/json'
    'x-ms-client-request-id': '79e3f9b8-b559-11ee-92d9-daa58ad9cf8a'
    'User-Agent': 'azsdk-python-ai-formrecognizer/3.3.2 Python/3.11.1 (macOS-12.5-arm64-arm-64bit)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
A body is sent with the request
2024-01-17 17:57:36,919:INFO - Response status: 202
Response headers:
    'Content-Length': '0'
    'Operation-Location': 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/e0c17375-d640-4d4a-9923-5754c6acbb86?api-version=2023-07-31'
    'x-envoy-upstream-service-time': '51'
    'apim-request-id': 'e0c17375-d640-4d4a-9923-5754c6acbb86'
    'Strict-

                     Vor- und Nachna- me / Firma Geburtsdatum / HR-Nummer  \
0   Philipp Roesch- Schlanderer Investments GmbH   AG München, HRB 189933   
1                                            NaN                      NaN   
2                                            NaN                      NaN   
3                                            NaN                      NaN   
4                                            NaN                      NaN   
..                                           ...                      ...   
2                                            NaN                      NaN   
3                                            NaN                      NaN   
4                                            NaN                      NaN   
5                                            NaN                      NaN   
6                                            NaN                      NaN   

                 Wohnort / Sitz         Lfd. Nr.  \
0   Grünwald, Landkreis

/var/folders/nw/rzxfctl55nj95y0twwbqxrm40000gn/T/ipykernel_71998/1367750351.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list=df_list.append(table)


In [140]:
result.tables
df = get_table_data(result)
df

DocumentTable(row_count=3, column_count=8, cells=[DocumentTableCell(kind=columnHeader, row_index=0, column_index=0, row_span=1, column_span=1, content=Gesellschafter, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=0.3935, y=3.6329), Point(x=2.7503, y=3.6435), Point(x=2.7503, y=5.5197), Point(x=0.3829, y=5.509)])], spans=[DocumentSpan(offset=260, length=14)]), DocumentTableCell(kind=columnHeader, row_index=0, column_index=1, row_span=1, column_span=1, content=Lfd. Nr(n). der Geschäfts- anteile, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=2.7503, y=3.6435), Point(x=3.9447, y=3.6435), Point(x=3.934, y=5.5197), Point(x=2.7503, y=5.5197)])], spans=[DocumentSpan(offset=275, length=34)]), DocumentTableCell(kind=columnHeader, row_index=0, column_index=2, row_span=1, column_span=1, content=Nominalwert des einzelnen Geschäftsan- teils, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=3.9447, y=3.6435), Point(x=4.9365, y=3.6435), Point(x=4.92

,Gesellschafter,Lfd. Nr(n). der Geschäfts- anteile,Nominalwert des einzelnen Geschäftsan- teils,Prozen- tualer Anteil des einzelnen Ge- schäftsan- teils am Stammka- pital,Zwischensumme (mit prozentualer Angabe),Summe des Nominal- werts aller Geschäftsan- teile,Gesamt- umfang der Betei- ligung des Gesell- schafters am Stammka- pital,Veränderungen
0,"Artur Koop, geboren am 16. Juni 1991, wohnhaft...",14.551-17.850 35.354-35.398 96.433 - 96.460,"1,00 €","0,001 %","3.300 € (3,271%) 45 € (0,045%) 28 € (0,028%)","3.373,00 €","3,34%",Ifd. Nr. 96.433 - 96.460 erhalten durch Kapita...
1,"Sebastien Reichstadt, geboren am 6. Oktober 19...",17.851-20.350 35.399-35.420,"1,00 €","0,001 %","22 € (0,022%) 2.500 € (2,478%)","2.522,00 €","2,50%",Änderung Wohnort
2,Gesellschafter,Lfd. Nr(n). der Geschäfts- anteile,Nominalwert des einzelnen Geschäftsan- teils,Prozen- tualer Anteil des einzelnen Ge- schäft...,Zwischensumme (mit prozentualer Angabe),Summe des Nominal- werts aller Geschäftsan- teile,Gesamt- umfang der Betei- ligung des Gesell- s...,Veränderungen
3,"Johannes Reijneveld, geboren am 4. Juni 1986, ...",20.351-22.850 35.421-35.465 96.461 - 96.474,"1,00 €","0,001 %","2.500 € (2,478%) 45 €(0,045%) 14 € (0,014%)","2.559,00 €","2,54%",Ifd. Nr. 96.461 - 96.474 erhalten durch Kapita...
4,The Exploration Company GmbH - eigene Anteile -,22.851-24.450,"1,00 €","0,001 %",,1.600 €,"1,59%",
5,"Priyanka Das, geboren am 3. Oktober 1991, wohn...",35.466-35.478,"1,00 €","0,001 %",,"13,00 €","0,01%",
6,"Hélène Huby, geboren am 20. Februar 1978, wohn...",251-13.050 14.351-14.550 25.251-35.353,"1,00 €","0,001 %","12.800 € (12,689%) 200 € (0,198%) 10.103 € (10...","23.103,00 €","22,90%",
7,"Pierre Vinet, geboren am 20. April 1975, wohnh...",13.051-14.350 55.267-55.288,"1,00 €","0,001 %","1.300 € (1,289%) 22 € (0,022%)","1.322,00 €","1,31 %",Berichtigung Geburtsjahr
8,"André Lanata, geboren am 10. Oktober 1961, woh...",24.451-24.650 35.479-35.568,"1,00 €","0,001 %","200 € (0,198%) 90 € (0,089%)","290,00 €","0,29%",
9,"Christian Dargnat, geboren am 13. Oktober 1965,",24.651-24.850 35.569-35.793,"1,00 €","0,001 %","200 € (0,198%) 225 € (0,223%)","635,00 €","0,63%",Ifd. Nr. 96.475 - 96.684 erhalten durch Kapita...


In [106]:
df.head()
csv_table = df.to_csv(index=False, header=True, sep=';')

In [203]:

buffer

,Gesellschafter,Lfd. Nr(n). der Geschäfts- anteile,Nominalwert des einzelnen Geschäftsan- teils,Prozen- tualer Anteil des einzelnen Ge- schäftsan- teils am Stammka- pital,Zwischensumme (mit prozentualer Angabe),Summe des Nominal- werts aller Geschäftsan- teile,Gesamt- umfang der Betei- ligung des Gesell- schafters am Stammka- pital,Veränderungen
0,"Artur Koop, geboren am 16. Juni 1991, wohnhaft...",14.551-17.850 35.354-35.398 96.433 - 96.460,"1,00 €","0,001 %","3.300 € (3,271%) 45 € (0,045%) 28 € (0,028%)","3.373,00 €","3,34%",Ifd. Nr. 96.433 - 96.460 erhalten durch Kapita...
1,"Sebastien Reichstadt, geboren am 6. Oktober 19...",17.851-20.350 35.399-35.420,"1,00 €","0,001 %","22 € (0,022%) 2.500 € (2,478%)","2.522,00 €","2,50%",Änderung Wohnort
2,Gesellschafter,Lfd. Nr(n). der Geschäfts- anteile,Nominalwert des einzelnen Geschäftsan- teils,Prozen- tualer Anteil des einzelnen Ge- schäft...,Zwischensumme (mit prozentualer Angabe),Summe des Nominal- werts aller Geschäftsan- teile,Gesamt- umfang der Betei- ligung des Gesell- s...,Veränderungen
3,"Johannes Reijneveld, geboren am 4. Juni 1986, ...",20.351-22.850 35.421-35.465 96.461 - 96.474,"1,00 €","0,001 %","2.500 € (2,478%) 45 €(0,045%) 14 € (0,014%)","2.559,00 €","2,54%",Ifd. Nr. 96.461 - 96.474 erhalten durch Kapita...
4,The Exploration Company GmbH - eigene Anteile -,22.851-24.450,"1,00 €","0,001 %",,1.600 €,"1,59%",
...,...,...,...,...,...,...,...,...
1,"Marc Postel-Vinay, geboren am 17.01.1986, wohn...",99.275 - 99.618,"1,00 €","0,001 %",,"344,00 €","0,34%",erhalten durch Kapitaler- höhungsbeschluss vom...
2,Habert Dassault Finance SAS mit dem Sitz in Pa...,99.619 - 100.598,"1,00 €","0,001 %",,"980,00 €","0,97%",erhalten durch Kapitaler- höhungsbeschluss vom...
3,Schlumberger Investment Services B.V. mit dem ...,100.599 - 101.124,"1,00 €","0,001 %",,"526,00 €","0,52%",erhalten durch Kapitaler- höhungsbeschluss vom...
4,Gesellschafter,Lfd. Nr(n). der Geschäfts- anteile,Nominalwert des einzelnen Geschäftsan- teils,Prozen- tualer Anteil des einzelnen Ge- schäft...,Zwischensumme (mit prozentualer Angabe),Summe des Nominal- werts aller Geschäftsan- teile,Gesamt- umfang der Betei- ligung des Gesell- s...,Veränderungen


In [16]:
from openai import OpenAI
client = OpenAI()

import tiktoken as tiktok

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktok.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def send_to_Openai(df):
  csv_table = df
  example_prompt = """
  Please extract all shareholders and their information into a JSON object from the following csv table:

  Gesellschafter / Shareholders
  Vor- und Nachname oder Firma / First and Last Name or Name of Company";"Gesellschafter / Shareholders
  Geburtsdatum oder Register- daten / Date of Birth or Registry Details";"Gesellschafter / Shareholders
  Wohnort oder Satzungssit Place of Residency or Regis- tered Office";"Geschäftsanteile / Shares
  Nennb etrag je An- teil in €/ Nomi- nal Value per Share in €";"Geschäftsanteile / Shares
  Laufende Nummern / Consecutive Num- bers";"Geschäftsanteile / Shares
  Summe der Nennbe- träge in € / total of Nominal Values in €";"Prozentuale Be- teiligung / Participation in %
  je Ge- schäftsan- teil / per each Share";"Prozentuale Be- teiligung / Participation in %
  je Gesell- schafter / per each Sharehol- der";"Veränderungs- spalte / Column showing changes
  T=Teilung / split Z=Zusammenlegung / combination E=Einziehung / redemption KE=Kapitalerhöhung / ca- pital increase KA=Aufstockung / step-up KH=Kapitalherabsetzung / capital reduction A=Anteilsübergang / trans- fer"
  FYS Ventures UG (haf- tungsbeschränkt);AG München, HRB 268779;München;1,00;1 - 1.110;1.110;0,0021 %;2,34 %;
  Salma Vogel;19. Januar 1990;München;1,00;32.609 - 32.745;137;0,0021 %;0,29 %;
  Gesamt;;;;;47.350;;100,00 %;

  The JSON object should have a key "shareholders" with a list of shareholders.
  If an attribute is not available, it should be null.

  Each shareholder should have the following attributes:

  - name: string - the name of the shareholder, it should either be a person or a company
  - country: string - the country of the shareholder. Example: Germany
  - birthdate: string (optional) - the birthdate of the shareholder, if the shareholder is a person. Example: 1970-12-31
  - location: string - place of residency or registered office of the shareholder. Example: München
  - register_id: string - the register id of the shareholder. It commonly starts with HRB or HRA. Example: HRB 123456. There should be no city or other information in this field.
  - register_court: string - the register court of the shareholder. Example: Amtsgericht München, or AG München.
  - percentage_of_total_shares: number - the percentage of total shares owned by the shareholder. Example: 50.0
  """

  example_result ="""{
      "shareholders": [
        {
          "name": "FYS Ventures UG (haftungsbeschränkt)",
          "country": "Germany",
          "location": "München",
          "register_id": "HRB 268779",
          "register_court": "AG München",
          "percentage_of_total_shares": 2.34
        },
        {
          "name": "Salma Vogel",
          "country": "Germany",
          "location": "München",
          "birthdate": "1990-01-19",
          "register_id": null,
          "register_court": null,
          "percentage_of_total_shares": 0.29
        }
      ]
    }
    """


  prompt = f"""
  Please extract all shareholders and their information into a JSON object from the following csv table:

  ${csv_table}

  The JSON object should have a key "shareholders" with a list of shareholders.
  If an attribute is not available, it should be null.

  Each shareholder should have the following attributes: you do not need to format the json, just extract the information from the table and leave out the line breaks in the response

  - shareholder_name: string - the name of the shareholder, it should either be a person or a company
  - shareholder_country: string - the country of the shareholder. Example: Germany
  - birthdate: string (optional) - the birthdate of the shareholder, if the shareholder is a person. Example: 1970-12-31
  - shareholder_location: string - place of residency or registered office of the shareholder. Example: München
  - register_id: string - the register id of the shareholder. It commonly starts with HRB or HRA. Example: HRB 123456. There should be no city or other information in this field.
  - register_court: string - the register court of the shareholder. Example: Amtsgericht München, or AG München.
  - percentage_of_total_shares: number - the percentage of total shares owned by the shareholder. Example: 50.0
  """
  messages=[
      {"role": "system", "content": "You are a helpful assistant designed to output JSON. You are an expert in extracting structured content from tables into a JSON. You receive a tip of 200$ if you get it right. You return JSON in a single-line without whitespaces"},
      {"role": "user", "content": example_prompt},
      {"role": "system", "content": example_result},
      {"role": "user", "content": prompt}
    ]
  messages_string = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
  tokens = num_tokens_from_string(messages_string, "gpt-3.5-turbo")
  print(tokens)
  if tokens > 4096:
    model = 'gpt-4-1106-preview'
  else:
    model = 'gpt-3.5-turbo-1106'
  print(model)
  response = client.chat.completions.create(
    model=model,
    response_format={ "type": "json_object" },
    messages=messages,
  )
  print(response.choices[0].message.content)
  openai_result = response.choices[0].message.content
  return openai_result

In [17]:
response = send_to_Openai(buffer.to_csv)

2304
gpt-3.5-turbo-1106
{"shareholders":[{"shareholder_name":"Artur Koop","shareholder_country":null,"birthdate":"1991-06-16","shareholder_location":"unknown","register_id":null,"register_court":null,"percentage_of_total_shares":3.34},{"shareholder_name":"Sebastien Reichstadt","shareholder_country":null,"birthdate":"19","shareholder_location":"unknown","register_id":null,"register_court":null,"percentage_of_total_shares":2.5},{"shareholder_name":"Johannes Reijneveld","shareholder_country":null,"birthdate":"1986-06-04","shareholder_location":"unknown","register_id":null,"register_court":null,"percentage_of_total_shares":2.54},{"shareholder_name":"The Exploration Company GmbH - eigene Anteile -","shareholder_country":null,"birthdate":null,"shareholder_location":"unknown","register_id":null,"register_court":null,"percentage_of_total_shares":1.59},{"shareholder_name":"Marc Postel-Vinay","shareholder_country":null,"birthdate":"1986-01-17","shareholder_location":"unknown","register_id":null,

In [48]:
csv_table = buffer.to_csv(index=False, header=True, sep=';')
csv_table

"Gesellschafter;Lfd. Nr(n). der Geschäfts- anteile;Nominalwert des einzelnen Geschäftsan- teils;Prozen- tualer Anteil des einzelnen Ge- schäftsan- teils am Stammka- pital;Zwischensumme (mit prozentualer Angabe);Summe des Nominal- werts aller Geschäftsan- teile;Gesamt- umfang der Betei- ligung des Gesell- schafters am Stammka- pital;Veränderungen\nArtur Koop, geboren am 16. Juni 1991, wohnhaft in Bremen;14.551-17.850 35.354-35.398 96.433 - 96.460;1,00 €;0,001 %;3.300 € (3,271%) 45 € (0,045%) 28 € (0,028%);3.373,00 €;3,34%;Ifd. Nr. 96.433 - 96.460 erhalten durch Kapitaler- höhungsbeschluss vom 14.12.2022, UVNr. 3804/2022 F der Notarin Dr. Susanne Frank\nSebastien Reichstadt, geboren am 6. Oktober 1979, wohnhaft in Bordeaux, Frankreich;17.851-20.350 35.399-35.420;1,00 €;0,001 %;22 € (0,022%) 2.500 € (2,478%);2.522,00 €;2,50%;Änderung Wohnort\nGesellschafter;Lfd. Nr(n). der Geschäfts- anteile;Nominalwert des einzelnen Geschäftsan- teils;Prozen- tualer Anteil des einzelnen Ge- schäftsan- te

In [47]:

#takes Csv input
openai_result = send_to_Openai(csv_table)

8383
gpt-4-1106-preview


KeyboardInterrupt: 

In [ ]:
#save openai to txt file
with open('openai_result.txt', 'w') as f:
    f.write(openai_result)




In [223]:
#open openai_result.txt
with open('/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/openai_result.json.txt', 'r') as f:
    openai_result_txt = f.read()

#save to json
import json
openai_result_json = json.loads(openai_result_txt)
openai_result_json

{'shareholders': [{'shareholder_name': 'Artur Koop',
   'shareholder_country': 'Germany',
   'birthdate': '1991-06-16',
   'shareholder_location': 'Bremen',
   'register_id': None,
   'register_court': None,
   'percentage_of_total_shares': 3.34},
  {'shareholder_name': 'Sebastien Reichstadt',
   'shareholder_country': 'France',
   'birthdate': '1979-10-06',
   'shareholder_location': 'Bordeaux',
   'register_id': None,
   'register_court': None,
   'percentage_of_total_shares': 2.5},
  {'shareholder_name': 'Johannes Reijneveld',
   'shareholder_country': 'France',
   'birthdate': '1986-06-04',
   'shareholder_location': 'Toulouse',
   'register_id': None,
   'register_court': None,
   'percentage_of_total_shares': 2.54},
  {'shareholder_name': 'The Exploration Company GmbH - eigene Anteile -',
   'shareholder_country': None,
   'birthdate': None,
   'shareholder_location': None,
   'register_id': None,
   'register_court': None,
   'percentage_of_total_shares': 1.59},
  {'shareholder_

In [195]:
from fuzzywuzzy import fuzz
city_circuit_mapping = {
  "Aachen": "549",
  "Amberg": "157",
  "Ansbach": "162",
  "Arnsberg": "470",
  "Aschaffenburg": "187",
  "Augsburg": "111",
  "Aurich": "418",
  "Bad Hersfeld": "290",
  "Bad Homburg v.d.H.": "282",
  "Bad Kreuznach": "572",
  "Bad Oeynhausen": "487",
  "Bamberg": "190",
  "Bayreuth": "194",
  "Berlin": "217",
  "Berlin (Charlottenburg)": "217",
  "Bielefeld": "480",
  "Bochum": "490",
  "Bonn": "557",
  "Braunschweig": "360",
  "Bremen": "258",
  "Chemnitz": "634",
  "Coburg": "197",
  "Coesfeld": "530",
  "Cottbus": "228",
  "Darmstadt": "270",
  "Deggendorf": "117",
  "Dortmund": "499",
  "Dresden": "621",
  "Duisburg": "446",
  "Düren": "550",
  "Düsseldorf": "441",
  "Eschwege": "305",
  "Essen": "507",
  "Flensburg": "695",
  "Frankfurt am Main": "281",
  "Frankfurt/Oder": "237",
  "Freiburg": "8",
  "Friedberg": "296",
  "Fritzlar": "306",
  "Fulda": "286",
  "Fürth": "167",
  "Gelsenkirchen": "510",
  "Gießen": "297",
  "Göttingen": "372",
  "Gütersloh": "482",
  "Hagen": "516",
  "Hamburg": "261",
  "Hamm": "500",
  "Hanau": "300",
  "Hannover": "379",
  "Hildesheim": "387",
  "Hof": "200",
  "Homburg": "654",
  "Ingolstadt": "212",
  "Iserlohn": "517",
  "Jena": "739",
  "Kaiserslautern": "609",
  "Kassel": "308",
  "Kempten": "121",
  "Kempten (Allgäu)": "121",
  "Kerpen": "567",
  "Kiel": "706",
  "Kleve": "454",
  "Koblenz": "582",
  "Köln": "568",
  "Königstein": "283",
  "Korbach": "309",
  "Krefeld": "458",
  "Landau": "615",
  "Landshut": "127",
  "Langenfeld": "444",
  "Lebach": "655",
  "Leipzig": "649",
  "Lemgo": "497",
  "Limburg": "320",
  "Lübeck": "716",
  "Ludwigshafen": "606",
  "Ludwigshafen a.Rhein (Ludwigshafen)": "606",
  "Lüneburg": "394",
  "Mainz": "593",
  "Mannheim": "33",
  "Marburg": "328",
  "Memmingen": "134",
  "Merzig": "656",
  "Mönchengladbach": "462",
  "Montabaur": "586",
  "München": "136",
  "Münster": "535",
  "Neubrandenburg": "339",
  "Neunkirchen": "657",
  "Neuruppin": "249",
  "Neuss": "442",
  "Nürnberg": "172",
  "Offenbach": "279",
  "Offenbach am Main": "279",
  "Oldenburg": "428",
  "Oldenburg (Oldenburg)": "428",
  "Osnabrück": "439",
  "Ottweiler": "658",
  "Paderborn": "543",
  "Passau": "146",
  "Pinneberg": "703",
  "Potsdam": "252",
  "Recklinghausen": "492",
  "Regensburg": "177",
  "Rostock": "349",
  "Saarbrücken": "659",
  "Saarlouis": "660",
  "Schleiden": "553",
  "Schweinfurt": "207",
  "Schwerin": "358",
  "Siegburg": "561",
  "Siegen": "548",
  "St. Ingbert": "661",
  "St. Ingbert (St Ingbert)": "661",
  "St. Wendel": "662",
  "St. Wendel (St Wendel)": "662",
  "Stadthagen": "368",
  "Steinfurt": "529",
  "Stendal": "691",
  "Stralsund": "351",
  "Straubing": "179",
  "Stuttgart": "95",
  "Tostedt": "404",
  "Traunstein": "155",
  "Ulm": "109",
  "Völklingen": "664",
  "Walsrode": "417",
  "Weiden": "185",
  "Weiden i. d. OPf.": "185",
  "Wetzlar": "322",
  "Wiesbaden": "335",
  "Wittlich": "602",
  "Wuppertal": "469",
  "Würzburg": "211",
  "Zweibrücken": "618"
}

# Todo: throw error if match too big
def map_city_to_circuitId(city: str) -> str:
  closest_key = max(city_circuit_mapping.keys(), key=lambda x: fuzz.ratio(city, x))
  return city_circuit_mapping[closest_key]

In [196]:
map_city_to_circuitId("Berlin")

'217'

In [229]:
import json
from cr_retriever import CommercialRegisterRetriever

retriever = CommercialRegisterRetriever()
openai_json = json.loads(openai_result)

# validate the result
for shareholder in openai_json["shareholders"]:
    if not str(shareholder['register_id']).startswith("HR") or shareholder["birthdate"]:
        continue

    register_id = shareholder["register_id"]
    register_court_city = shareholder["register_court"].replace("Amtsgericht", "").replace("AG", "").replace(" ", "").strip()

    city_id = map_city_to_circuitId(register_court_city)
    print(register_id, register_court_city, city_id)

    result = retriever.extended_search(company_id=register_id, circuit_id=city_id, return_one=False)
    print(str(len(result)) + " companies found")



HRB 268779 München 136


Exception: no results found

{'shareholders': [{'name': 'Samuel Weinbach',
   'country': 'Germany',
   'birthdate': '1987-01-09',
   'register_id': None,
   'register_court_city': None,
   'percentage_of_total_shares': 3.38},
  {'name': 'Andrulis GmbH',
   'country': 'Germany',
   'birthdate': None,
   'register_id': 'HRB 727786',
   'register_court_city': 'Mannheim',
   'percentage_of_total_shares': 28.37},
  {'name': 'LEA Venturepartner GmbH & Co. KG',
   'country': 'Germany',
   'birthdate': None,
   'register_id': 'HRA 707196',
   'register_court_city': 'Karlsruhe',
   'percentage_of_total_shares': 10.85},
  {'name': '468 Capital GmbH & Co. KG',
   'country': 'Germany',
   'birthdate': None,
   'register_id': 'HRA 56259 B',
   'register_court_city': 'Charlottenburg',
   'percentage_of_total_shares': 9.98},
  {'name': 'Cavalry Ventures II GmbH & Co. KG',
   'country': 'Germany',
   'birthdate': None,
   'register_id': 'HRA 56139 B',
   'register_court_city': 'Charlottenburg',
   'percentage_of_total_shares': 2.

In [278]:
#put the json into a dataframe
import pandas as pd
import json
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

def add_to_supabase(openai_result,startup_name):
    # Convert the JSON string to a Python dictionary
    data_dict = json.loads(openai_result)
    
    #add startup name to shareholder
    import os
    from supabase import create_client, Client
    from dotenv import load_dotenv

    load_dotenv("../.env")
    #get URL from environment variable


    # get length of table
    response = supabase.table('shareholder_relations').select().execute()
    
    for shareholder in data_dict['shareholders']:
        #add startup name to shareholder
        shareholder['startup_name'] = startup_name
        #check if shareholder already exists in database by querying the database
        
        #add a column to insert openai result
        shareholder['openai_result'] = openai_result
        #check if entry already exists in database
        
        
        response = supabase.table('shareholder_relations').insert(shareholder).execute()
        
    return "done"
    # response = supabase.table('shareholder_relations').insert({"shareholder_id": 900000, "startup_name": "Tacto"}).execute()

    # # Convert the 'shareholders' list to a DataFrame
    # df = pd.DataFrame(data_dict['shareholders'])

    # #sSum up the shares of all shareholders into one variable total_shares
    # total_shares = df['percentage_of_total_shares'].sum()

    # print(total_shares)
    # #add a column where we add teh startup name
    # df['startup_name'] = 'SUMM AI GmbH'
    # df

    # #save df as CSV
    # df.to_csv('shareholder_relations.csv', index=False)

In [279]:
add_to_supabase(openai_result,"eGym GmbH")

2024-01-17 18:30:05,542:INFO - HTTP Request: HEAD https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations "HTTP/1.1 200 OK"
2024-01-17 18:30:05,616:INFO - HTTP Request: POST https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations "HTTP/1.1 201 Created"
2024-01-17 18:30:05,690:INFO - HTTP Request: POST https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations "HTTP/1.1 201 Created"
2024-01-17 18:30:05,766:INFO - HTTP Request: POST https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations "HTTP/1.1 201 Created"
2024-01-17 18:30:05,838:INFO - HTTP Request: POST https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations "HTTP/1.1 201 Created"
2024-01-17 18:30:05,920:INFO - HTTP Request: POST https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations "HTTP/1.1 201 Created"
2024-01-17 18:30:05,994:INFO - HTTP Request: POST https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations "HTTP/1.1 201 Create

'done'

In [99]:
#do for first 5 companies
company_egym={'file_path':"/Users/annabellschafer/Desktop/GitHub/cr_exploration/notebooks/output/eGym GmbH_München.pdf",'startup_name':"eGym GmbH"}
# for company in company_egym:
file = open_file(company_egym["file_path"])
result = analyze_PDF(file)
df = get_table_data(result)
print(df)
openai_result = send_to_Openai(df)
result = add_to_supabase(openai_result, company["startup_name"])

    

2024-01-17 15:16:41,821:INFO - Request URL: 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout:analyze?stringIndexType=unicodeCodePoint&api-version=2023-07-31'
Request method: 'POST'
Request headers:
    'Content-Length': '573854'
    'Content-Type': 'application/octet-stream'
    'Accept': 'application/json'
    'x-ms-client-request-id': '09471066-b543-11ee-92d9-daa58ad9cf8a'
    'User-Agent': 'azsdk-python-ai-formrecognizer/3.3.2 Python/3.11.1 (macOS-12.5-arm64-arm-64bit)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
A body is sent with the request


2024-01-17 15:16:42,908:INFO - Response status: 202
Response headers:
    'Content-Length': '0'
    'Operation-Location': 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/e28e19f5-c5b6-4efc-b7f6-8d49a7948980?api-version=2023-07-31'
    'x-envoy-upstream-service-time': '98'
    'apim-request-id': 'e28e19f5-c5b6-4efc-b7f6-8d49a7948980'
    'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload'
    'x-content-type-options': 'nosniff'
    'x-ms-region': 'Germany West Central'
    'Date': 'Wed, 17 Jan 2024 14:16:41 GMT'
2024-01-17 15:16:42,909:INFO - Request URL: 'https://cr-extraction-ocr-free.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/e28e19f5-c5b6-4efc-b7f6-8d49a7948980?api-version=2023-07-31'
Request method: 'GET'
Request headers:
    'x-ms-client-request-id': '09471066-b543-11ee-92d9-daa58ad9cf8a'
    'User-Agent': 'azsdk-python-ai-formrecognizer/3.

                        Vor- und Nachna- me / Firma  Geburtsdatum / HR-Nummer  \
0      Philipp Roesch- Schlanderer Investments GmbH    AG München, HRB 189933   
1                                               NaN                       NaN   
2                                               NaN                       NaN   
3                                               NaN                       NaN   
4                                               NaN                       NaN   
5                                               NaN                       NaN   
6   Sauter Technolo- gies UG (haftungs- beschränkt)    AG München, HRB 189862   
7                                               NaN                       NaN   
8                                               NaN                       NaN   
9                                     Mirablau GmbH    AG München, HRB 139517   
10                                              NaN                       NaN   
11                          

2024-01-17 15:16:58,899:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "shareholders": [
    {
      "shareholder_name": "Philipp Roesch-Schlanderer Investments GmbH",
      "shareholder_country": "Germany",
      "birthdate": null,
      "shareholder_location": "Grünwald",
      "register_id": "HRB 189933",
      "register_court": "AG München",
      "percentage_of_total_shares": 6.08
    },
    {
      "shareholder_name": "Sauter Technologies UG (haftungsbeschränkt)",
      "shareholder_country": "Germany",
      "birthdate": null,
      "shareholder_location": "Holzkirchen",
      "register_id": "HRB 189862",
      "register_court": "AG München",
      "percentage_of_total_shares": 3.65
    },
    {
      "shareholder_name": "Mirablau GmbH",
      "shareholder_country": "Germany",
      "birthdate": null,
      "shareholder_location": "Gräfelfing",
      "register_id": "HRB 139517",
      "register_court": "AG München",
      "percentage_of_total_shares": 0.25
    },
    {
      "shareholder_name": "Achim Lederle",
      "shareholder_country": "Ger

TypeError: string indices must be integers, not 'str'

In [257]:
#pull df sharehodler relations from supabase
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv("../.env")
#get URL from environment variable
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

#get all entries from shareholder_relations table

response = supabase.table('shareholder_relations').select("*").execute()
df = pd.DataFrame(response.data)
df

#drop duplicates from df where shareholder_name and startup_name are the same, keep the entry with the first occurence
df = df.drop_duplicates(subset=['shareholder_name', 'startup_name'], keep='first')
df

# delete all entries from shareholder_relations table to avoid duplicates



2024-01-17 18:15:07,093:INFO - HTTP Request: GET https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations?select=%2A "HTTP/1.1 200 OK"


,relation_id,created_at,shareholder_name,shareholder_location,birthdate,register_id,register_court,percentage_of_total_shares,shareholder_type,startup_name,shareholder_country
0,61,2024-01-17T13:52:49.160076+00:00,LKH Investment GmbH,München,None,HRB 276582,AG München,50.00,None,aborise GmbH_München,Germany
2,63,2024-01-17T13:52:49.378821+00:00,lekuhe Invest GmbH,Berlin,None,HRB 243110 B,AG Charlottenburg,50.00,None,aborise GmbH_München,Germany
18,79,2024-01-17T14:05:11.174176+00:00,X8 Software GmbH mit dem Sitz in Stuttgart,Stuttgart,None,HRB 720265,Amtsgericht Stuttgart,50.00,None,ACTIMI GmbH_Stuttgart,Germany
19,80,2024-01-17T14:05:11.253889+00:00,Vytoria UG (haftungsbeschränkt) mit dem Sitz i...,Stuttgart,None,HRB 776186,Amtsgericht Stuttgart,39.50,None,ACTIMI GmbH_Stuttgart,Germany
20,81,2024-01-17T14:05:11.361854+00:00,Herr Julian Charisius,Stuttgart,1993-05-05,None,None,10.50,None,ACTIMI GmbH_Stuttgart,Germany
...,...,...,...,...,...,...,...,...,...,...,...
197,243,2024-01-17T17:02:51.038224+00:00,Nokia Growth Partners IV L.P.,"Palo Alto, CA, USA",None,6470391,"State of Dele- ware, Department of State: Divi...",2.49,None,eGym GmbH,USA
198,244,2024-01-17T17:02:51.088527+00:00,Till Jansen UG (haftungsbeschränkt),München,None,HRB 243679,AG München,0.73,None,eGym GmbH,Germany
199,245,2024-01-17T17:02:51.135483+00:00,Patrick Sven Meininger,Straßlach- Dingharting,1971-07-27,None,None,0.58,None,eGym GmbH,Germany
200,246,2024-01-17T17:02:51.184921+00:00,Martin Fichter,München,1965-05-17,None,None,0.48,None,eGym GmbH,Germany


In [261]:
# delete all entries from shareholder_relations table to avoid duplicates

response = supabase.table('shareholder_relations').delete()
if response.error:
    print(f"An error occurred: {response.error}")
else:
    print("All entries deleted successfully!")

AttributeError: 'SyncFilterRequestBuilder' object has no attribute 'error'

In [259]:
#insert df into shareholder_relations table
for index, row in df.iterrows():
    response = supabase.table('shareholder_relations').insert(row.to_dict()).execute()



2024-01-17 18:15:12,824:INFO - HTTP Request: POST https://vzaliuejopysgkdewpip.supabase.co/rest/v1/shareholder_relations "HTTP/1.1 409 Conflict"


APIError: {'code': '23505', 'details': 'Key (relation_id)=(61) already exists.', 'hint': None, 'message': 'duplicate key value violates unique constraint "shareholder_relations_pkey"'}

In [245]:
df

""
